In [21]:
#lol24
from bs4 import BeautifulSoup
import requests
import os

In [22]:
TOPICS_TO_SCRAP = {
    "dramaty": 24,
    "erotyczne": 165,
    "fantasy": 16,
    "felietony": 13,
    "historyczne": 1,
    "horrory": 12,
    "inne": 36,
    "smieszne": 10,
    "kryminalne": 11,
    "milosne": 119,
    "obyczajowe": 14,
    "poezja": 28,
    "przygodowe": 21,
    "sci-fi": 4,
    "sny": 6,
    "szkolne": 4,
    "thrillery": 2
}
CORE_URL = "https://lol24.com"
CORE_DIR = "LOL24Dataset"

In [27]:
def get_page_url(topic: str, page_number: int):
    page_url = f"{CORE_URL}/opowiadania/{topic}"
    if page_number != 1:
        page_url = f"{page_url}-{page_number}"
    return page_url

def get_html_tags_with_stories(page_url: str):
    page = requests.get(page_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup.find_all('h2')[:14]

def download_stories(html_tags_with_stories, output_dir):
    for tag in html_tags_with_stories:
        subpage_url = f"{CORE_URL}{tag.a['href']}"
        story_id = subpage_url.split("-")[-1]
        subpage = requests.get(subpage_url)
        subpage_soup = BeautifulSoup(subpage.content, 'html.parser')
        story_text = subpage_soup.find_all("p")[0].text
        try:
            author = subpage_soup.find_all("p")[1].find_all("span")[0].text
            path = os.path.join(output_dir, author)
        except Exception:
            continue
        try:
            if not os.path.exists(path):
                os.makedirs(path)
            path = f"{path}/{story_id}.txt"
            with open(path, "w", encoding="utf-8") as file:
                file.write(story_text)
        except Exception:
            print(f"Error with author {author}. Skipping...")

def scrap(topic: str, no_pages: int, output_dir: str):
    if not os.path.exists(CORE_DIR):
        os.makedirs(CORE_DIR)
    output_dir = os.path.join(CORE_DIR, output_dir)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for i in range(1, no_pages+1):
        page_url = get_page_url(topic, i)
        html_tags_with_stories = get_html_tags_with_stories(page_url)
        download_stories(html_tags_with_stories, output_dir)
        

In [ ]:
for topic, no_pages in TOPICS_TO_SCRAP.items():
    print(f"Scrapping {topic}")
    scrap(topic, no_pages, topic)